In [ ]:
import os
import cv2
import pandas as pd
from shapely.geometry import Polygon, mapping
from osgeo import gdal
import rasterio
import geopandas as gpd
import numpy as np
import tqdm
import rasterio.mask
import torch
import math
dim=700

In [ ]:
def correct_orientation(img_rgb,dim):
#     img_rgb=cv2.imread(img_rgb)
    img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)
    img_gray = cv2.resize(img_gray,(dim,dim))
    template = cv2.imread("pattern1.png",0)
#     template=cv2.bitwise_not(template)
    w, h = template.shape[::-1]
    image_ro=img_gray.copy()
    angulo=0
    an=[]
    le=0
    angulo_f=0
    for i in range(0,90,5):
        angulo=i
        M = cv2.getRotationMatrix2D((dim//2,dim//2), angulo, 1)
        image_ro = cv2.warpAffine(img_gray, M, (dim,dim))
        res = cv2.matchTemplate(image_ro,template,cv2.TM_CCOEFF_NORMED)
        threshold =.5
        loc = np.where( res >= threshold)
        com=len(loc[0])
        if com>0:
            an.append(angulo)
            if le<com:
                le=com
                angulo_f=angulo
    return angulo_f
    

In [ ]:
def verificacion(im):
    hsv=cv2.cvtColor(im,cv2.COLOR_BGR2HSV)
    mask=cv2.inRange(hsv,(12,30,0),(160,232,160))
    verde=int((np.sum(mask)/256**2/255)*100)
    return verde 

In [ ]:
def vector2xy(vector,dim=700,nameimg="image",angle=0):
    s=[]
    for v in vector:
        str_v=(str(v).replace("tensor(","").replace("[","").replace("]","").replace(".)","").replace(")","").replace("(","").replace("']","").replace("'","").strip().split(","))
        h,w=dim,dim
        x1 = int( float(str_v[1]) * w )
        y1 = int( float(str_v[2]) * h )
        xw = int( float(str_v[3]) * w /2)
        yw = int( float(str_v[4]) * h /2)
        start_point_im = ((x1 - xw), (y1 - yw))
        end_point_im   = ((x1 + xw), (y1 + yw))
        start_point_100 = ((x1 - xw)/w, (y1 - yw)/h)
        end_point_100   = ((x1 + xw)/w, (y1 + yw)/h)
        area=xw*yw
        if str(str_v[0])=="0":
            tipo="casa"
        else:
            tipo="terreno"
        if int(xw)!=0 and int(yw)!=0 and (xw/yw<=3  and yw/xw<=3):
            s.append([tipo,start_point_im,end_point_im,start_point_100,end_point_100,area,nameimg])
    df_cache=pd.DataFrame(s,columns=["Tipo","start_point_im","end_point_im","start_point_100","end_point_100","area","imagen"])
#     print(np.min(df_cache["area"]),np.max(df_cache["area"]))
    try:
        df_cache["area"]=[int((x-np.min(df_cache["area"]))/(np.max(df_cache["area"])-np.min(df_cache["area"]))*100) for x in df_cache["area"]]
    except:
        pass
    df_cache.drop_duplicates().reset_index(drop=True,inplace=True)
#   
    return df_cache

In [ ]:
def imshow_detect(df_cache,imagen_n,nameimg="image"):
    for i in range(len(df_cache)):
            if df_cache["Tipo"][i]=="casa":
                cv2.rectangle(imagen_n,df_cache["start_point_im"][i],df_cache["end_point_im"][i],(0,255-int(df_cache.loc[i,"area"]),0),2)
            else:
                cv2.rectangle(imagen_n,df_cache["start_point_im"][i],df_cache["end_point_im"][i],(0,0,255),2)
#     imagen_n=cv2.resize(imagen_n,(1024,1024))
    cv2.imshow(nameimg,imagen_n)
    cv2.waitKey()
    cv2.destroyAllWindows()

In [ ]:
def rotacion_detect(startpoint,endpoint,angle,proyecciones,dim=700):
    min_y,min_x,z=np.min((proyecciones,proyecciones),axis=1)[0]
    max_y,max_x,z=np.max((proyecciones,proyecciones),axis=1)[0]
    min_y,min_x,max_y,max_x,proyecciones
    tipos=["casa","terreno"]
    y1,x1=startpoint
    y2,x2=endpoint
    x1,y1=x1*2-1,y1*2-1
    x2,y2=x2*2-1,y2*2-1
    angle=angle*math.pi/180
    #x_p, y_p son los puntos de un rectangulo en el orden inverso al manecillas del reloj
    x1p=max_x-((x1*math.cos(angle)-y1*math.sin(angle)+1)/2)*(max_x-min_x)
    y1p=min_y+((x1*math.sin(angle)+y1*math.cos(angle)+1)/2)*(max_y-min_y)
    x2p=max_x-((x2*math.cos(angle)-y1*math.sin(angle)+1)/2)*(max_x-min_x)
    y2p=min_y+((x2*math.sin(angle)+y1*math.cos(angle)+1)/2)*(max_y-min_y)
    x3p=max_x-((x2*math.cos(angle)-y2*math.sin(angle)+1)/2)*(max_x-min_x)
    y3p=min_y+((x2*math.sin(angle)+y2*math.cos(angle)+1)/2)*(max_y-min_y)
    x4p=max_x-((x1*math.cos(angle)-y2*math.sin(angle)+1)/2)*(max_x-min_x)
    y4p=min_y+((x1*math.sin(angle)+y2*math.cos(angle)+1)/2)*(max_y-min_y)
    
    return Polygon(((y1p,x1p),(y2p,x2p),(y3p,x3p),(y4p,x4p),(y1p,x1p)))

In [ ]:
os.chdir("C:/Users/ASUS/Inteligencia_Artificial/yolov7")
from detect_Alberto_v3 import modelo
Modelo=modelo(weights="Modelos/best_Hector_24.pt")

In [ ]:
raster=r"C:\Users\ASUS\Desktop\3naucalpan.tif"

gdal_interpeter = gdal.Open(raster)
width = gdal_interpeter.RasterXSize
height = gdal_interpeter.RasterYSize
coordenadas_gdal = gdal_interpeter.GetGeoTransform()
minx = coordenadas_gdal[0]
miny = coordenadas_gdal[3] + width*coordenadas_gdal[4] + height*coordenadas_gdal[5] 
maxx = coordenadas_gdal[0] + width*coordenadas_gdal[1] + height*coordenadas_gdal[2]
maxy = coordenadas_gdal[3] 
minx,maxx,miny,maxy,"W",maxx-minx,"H",maxy-miny

In [ ]:
src_raster_path = raster
src=rasterio.open(src_raster_path)
H,W=src.shape
src.crs

In [ ]:
dim=624
alto=int(np.floor(H/dim))
ancho=int(np.floor(W/dim))
alto,ancho,alto*ancho
import time
nombres=[]
imshow=False
result=pd.DataFrame()
casas=[]
terreno=[]
angulosget=[]
with tqdm.tqdm(total=alto*ancho) as pbar:
    for j in range(ancho):#ancho
        for i in (range(alto)):#alto
            # j=1
            t0 = time.time()
            label=raster.replace("\\","/").split("/")[-1][:-4]+"_"
            nameimg=label.lower()+str(i)+"_"+str(j)
            nombres.append(nameimg)
            cuadro=[]
            for k in range(2):
                for l in range(2):
                    cuadro.append((minx+(maxx-minx)/ancho*(j+k),
                                   maxy-(maxy-miny)/alto*(i+l),
                                   0.0))
            cuadro=[cuadro[0],cuadro[1],cuadro[3],cuadro[2],cuadro[0]]
            shapes=[{"type":'Polygon','coordinates':[cuadro]}]
            vector=[]
            array, out_transform = rasterio.mask.mask(src, shapes, crop=True)
            four_images=[array[2],array[1],array[0]]
            imagen_n = np.stack(four_images, axis=-1)
            angulo_f=0
            anterior=0
            actual=1
            vector1=[]
            angulo=0
            le=0
            angulo_1=correct_orientation(imagen_n,dim)
            for i in range(-2,2,1):
                t1 = time.time()
                angulo=angulo_1+i
                anterior=len(vector)
                image_ro=imagen_n.copy()
                M = cv2.getRotationMatrix2D((dim//2,dim//2), angulo, 1)
                image_ro = cv2.warpAffine(image_ro, M, (dim,dim))
                t2 = time.time()
                with torch.no_grad():
                    vector1=Modelo.detect(opt_source="cache1.png",opt_conf_thres=0.05,imagen_s=image_ro)
                t3 = time.time()
                actual=len(vector)
                if len(vector1)>le:
                    le=len(vector1)
                    vector=vector1
                    imagen_n1=image_ro
                    angulo_f=angulo
                t4 = time.time()
            angulo=angulo_f
            angulosget.append(angulo)
            proyecciones=shapes[0].get("coordinates")[0][:-1]
            df_cache=vector2xy(vector,dim=dim,nameimg=nameimg)
            
            for i in (range(len(df_cache))):
                if df_cache["Tipo"][i]=="casa":
                    casas.append(rotacion_detect(df_cache.loc[i,"start_point_100"], df_cache.loc[i,"end_point_100"],-angulo,proyeciones=proyecciones))
                else:
                    terreno.append(rotacion_detect(df_cache.loc[i,"start_point_100"], df_cache.loc[i,"end_point_100"],-angulo,proyeciones=proyecciones))
            if imshow:
                imshow_detect(df_cache,imagen_n)
            t4 = time.time()
#             print(t1-t0,t2-t1,t3-t2,t4-t3,t4-t0)
            
            pbar.update(1)
#             break



In [ ]:
gdf_casas=gpd.GeoDataFrame(geometry=casas,crs=src.crs)
gdf_casas.set_crs=src.crs
gdf_casas.to_file(r"C:/Users/ASUS/salida/shape/new_casas.shp")
gdf_casas["area"]=gdf_casas.area
gdf_casas=gdf_casas[(gdf_casas["area"]<=gdf_casas["area"].mean()*1.5)&(gdf_casas["area"]>45)]
gdf_casas.to_file(r"C:/Users/ASUS/salida/shape/new_casas_t.shp")
gdf_terreno=gpd.GeoDataFrame(geometry=terreno,crs=src.crs)
gdf_terreno.set_crs=src.crs
gdf_terreno.to_file(r"C:/Users/ASUS/salida/shape/new_terreno.shp")